In [1]:
cd drive/My\ Drive/concrete_crack

/content/drive/My Drive/concrete_crack


In [2]:
import numpy as np
import keras.utils
from keras.layers import Input, Dense, Flatten
from keras.models import load_model, Model
from keras.layers import Conv2D, UpSampling2D, AveragePooling2D
from keras.layers import LeakyReLU, Reshape
from keras.datasets import mnist
from keras.models import save_model
from keras import backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython import display
import time

Using TensorFlow backend.


In [0]:
import os

In [0]:
def generateTheta(L,endim):
    # This function generates L random samples from the unit `ndim'-u
    theta=[w/np.sqrt((w**2).sum()) for w in np.random.normal(size=(L,endim))]
    return np.asarray(theta)
def generateZ(batchsize):
    # This function generates 2D samples from a `circle' distribution in 
    # a 2-dimensional space
    r=np.random.uniform(size=(batchsize))
    theta=2*np.pi*np.random.uniform(size=(batchsize))
    x=r*np.cos(theta)
    y=r*np.sin(theta)
    z_=np.array([x,y]).T
    return z_

In [0]:
L = 50
batchsize = 100
endim = 2

In [5]:
img = Input(shape = (28,28,1))
x = Conv2D(16, (3,3), padding = 'same')(img)
x = LeakyReLU(alpha = 0.2)(x)
x = Conv2D(16, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = AveragePooling2D((2,2), padding = 'same')(x) #(none, 14, 14, 16)

x = Conv2D(32, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Conv2D(32, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = AveragePooling2D((2,2), padding = 'same')(x) #(none, 7, 7, 32)

x = Conv2D(64, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Conv2D(64, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = AveragePooling2D((2,2), padding = 'same')(x) #(none, 4, 4, 64)

x = Flatten()(x) #(none, 4*4*64)
x = Dense(128, activation = 'relu')(x)
encoded = Dense(endim)(x)

encoder = Model(inputs = [img], outputs = [encoded])
encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 28, 28, 16)        0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        4640

In [6]:
embedd = Input(shape = (endim, ))
x = Dense(128)(embedd)
x = Dense(4*4*64, activation = 'relu')(x)
x = Reshape((4,4,64))(x)
x = UpSampling2D((2,2))(x) #(none, 8, 8, 64)

x = Conv2D(64, (3,3), padding='same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Conv2D(64, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = UpSampling2D((2,2))(x) #(none, 16, 16, 64)

x = Conv2D(64, (3,3), padding = 'valid')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Conv2D(64, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = UpSampling2D((2,2))(x) #(none, 28, 28, 64)

x = Conv2D(32, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
x = Conv2D(32, (3,3), padding = 'same')(x)
x = LeakyReLU(alpha = 0.2)(x)
decoded = Conv2D(1, (3,3), padding = 'same', activation = 'sigmoid')(x)

decoder = Model(inputs = [embedd], outputs = [decoded])
decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               384       
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              132096    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 64)          0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 8, 8, 64)          0   

In [0]:
theta=K.variable(generateTheta(L,endim)) #Define a Keras Variable for \theta_ls
z=K.variable(generateZ(batchsize)) #Define a Keras Variable for samples of z

In [8]:
encoded_image = encoder(img)
decoded_image = decoder(encoded_image)
autoencoder = Model(inputs = [img], outputs = [decoded_image])
autoencoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
model_1 (Model)              (None, 2)                 203250    
_________________________________________________________________
model_2 (Model)              (None, 28, 28, 1)         308193    
Total params: 511,443
Trainable params: 511,443
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Let projae be the projection of the encoded samples
projae=K.dot(encoded_image,K.transpose(theta))
# Let projz be the projection of the $q_Z$ samples
projz=K.dot(z,K.transpose(theta))
# Calculate the Sliced Wasserstein distance by sorting 
# the projections and calculating the L2 distance between
W2=(tf.nn.top_k(tf.transpose(projae),k=batchsize).values-
    tf.nn.top_k(tf.transpose(projz),k=batchsize).values)**2

In [0]:
proj_encoded_transposed = tf.nn.top_k(tf.transpose(projae), k = batchsize).values
proj_z_transposed = tf.nn.top_k(tf.transpose(projz), k = batchsize).values
w2 = (proj_encoded_transposed - proj_z_transposed)**2
w2weight = K.variable(10.0)
w2loss = w2weight * K.mean(w2)

In [0]:
crossEntropy = K.binary_crossentropy(K.flatten(img), K.flatten(decoded_image))
crossEntropyLoss = (1.0) * K.mean(crossEntropy)

In [0]:
l1Loss = (1.0) * K.mean(K.abs(K.flatten(img) - K.flatten(decoded_image)))

In [0]:
total_loss = w2loss + crossEntropyLoss + l1Loss

In [0]:
autoencoder.add_loss(total_loss)

In [15]:
autoencoder.compile(optimizer = 'rmsprop', loss = '')

/usr/local/lib/python3.6/dist-packages/keras/engine/training_utils.py:819: UserWarning: Output model_2 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to model_2.
  'be expecting any data to be passed to {0}.'.format(name))


In [16]:
cd data_28

/content/drive/My Drive/concrete_crack/data_28


In [17]:
ls

rescaled_negative_data.npy  X_test_negative.npy
rescaled_positive_data.npy  X_test_positive_balanced.npy
X_test_balanced.npy         X_test_positive_imbalanced.npy
X_test_imbalanced.npy       X_train.npy


In [0]:
X_train = np.load('X_train.npy', allow_pickle=True)

In [0]:
X_test_balanced = np.load('X_test_balanced.npy', allow_pickle=True)

In [0]:
X_test_imbalanced = np.load('X_test_imbalanced.npy', allow_pickle=True)

In [21]:
history = autoencoder.fit(X_train, None, validation_split=0.2, epochs = 100, batch_size = 100)

Train on 12800 samples, validate on 3200 samples
Epoch 1/100
12800/12800 [==============================] - 12s 966us/step - loss: 2.3875 - val_loss: 1.8791
Epoch 2/100
12800/12800 [==============================] - 7s 544us/step - loss: 1.6760 - val_loss: 1.3526
Epoch 3/100
12800/12800 [==============================] - 7s 546us/step - loss: 1.1723 - val_loss: 2.6715
Epoch 4/100
12800/12800 [==============================] - 7s 544us/step - loss: 1.0465 - val_loss: 1.0855
Epoch 5/100
12800/12800 [==============================] - 7s 545us/step - loss: 0.8928 - val_loss: 0.8025
Epoch 6/100
12800/12800 [==============================] - 7s 543us/step - loss: 0.8279 - val_loss: 0.7190
Epoch 7/100
12800/12800 [==============================] - 7s 545us/step - loss: 0.7934 - val_loss: 0.8929
Epoch 8/100
12800/12800 [==============================] - 7s 542us/step - loss: 0.7824 - val_loss: 0.7488
Epoch 9/100
12800/12800 [==============================] - 7s 540us/step - loss: 0.7706 - val_

In [22]:
pwd

'/content/drive/My Drive/concrete_crack/data_28'

In [23]:
cd ..

/content/drive/My Drive/concrete_crack


In [0]:
os.mkdir('model1')

In [26]:
cd model1/

/content/drive/My Drive/concrete_crack/model1


In [0]:
save_model(encoder,filepath='encoder.h5')
save_model(decoder,filepath='decoder.h5')
save_model(autoencoder,filepath='autoencoder.h5')

In [0]:
np.save('model1_history', history.history, allow_pickle=True)

In [0]:
np.save('training_loss', history.history['loss'], allow_pickle=True)
np.save('validation_loss', history.history['val_loss'], allow_pickle=True)